In [15]:
import requests
from bs4 import BeautifulSoup
from glob import glob
import os
import fnmatch
import urllib
import os
import fnmatch
import unicodedata
import lxml.html
import xml
import re
import pandas as pd 
from bs4 import BeautifulSoup
from concurrent import futures

# Get cable files

In [6]:
resp = requests.get('https://file.wikileaks.org/file/cablegate/')
def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

t = listFD('https://file.wikileaks.org/file/cablegate/')

def fetch(a_href):
    output_file = a_href.split('/')[-1]
    urllib.urlretrieve(
        a_href,
        filename='data/compressed_files/' + output_file
    )

In [ ]:
# with futures.ThreadPoolExecutor(max_workers=4) as executor:
    # executor.map(fetch, t[1:])
map(fetch, t[1:])

# Read and Parse Them

In [3]:
import pylzma

In [4]:
import py7zlib
class SevenZFile(object):
    @classmethod
    def is_7zfile(cls, filepath):
        '''
        Class method: determine if file path points to a valid 7z archive.
        '''
        is7z = False
        fp = None
        try:
            fp = open(filepath, 'rb')
            archive = py7zlib.Archive7z(fp)
            n = len(archive.getnames())
            is7z = True
        finally:
            if fp:
                fp.close()
        return is7z

    def __init__(self, filepath):
        fp = open(filepath, 'rb')
        self.archive = py7zlib.Archive7z(fp)

    def extractall(self, path):
        for name in self.archive.getnames():
            outfilename = os.path.join(path, name)
            outdir = os.path.dirname(outfilename)
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            outfile = open(outfilename, 'wb')
            outfile.write(self.archive.getmember(name).read())
            outfile.close()

In [8]:
compressed_files = os.listdir('data/compressed_files/')

for fn in compressed_files:
    folder = fn.replace('.7z', '')
    if not os.path.exists('data/extracted_files/' + folder):
        print folder
        f = SevenZFile('data/compressed_files/' + fn)
        try:
            f.extractall('data/extracted_files/' + folder)
        except ValueError as e:
            pass

# HTML Parsing

In [16]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = unicode(cleantext , encoding='utf-8') #.encode('ascii', )
    cleantext = unicodedata.normalize('NFKD', cleantext).encode('ascii','ignore')
    cleantext = cleantext.replace('&#x000A;', '').replace('\\', '').replace('\n', '')
    return cleantext

## get list of extracted files
extracted_files = os.listdir('data/extracted_files')
matches = []
for extracted_fn in extracted_files:
    for root, dirnames, filenames in os.walk('data/extracted_files/{}/cable/'.format(extracted_fn)):
        for filename in fnmatch.filter(filenames, '*.html'):
            matches.append(os.path.join(root, filename))

In [ ]:
# strip HTML
output = {}
import time
now = time.time()
for i, fn in enumerate(matches):
    if i % 10000 == 0: 
        print 'completed %d in %f' % (i, time.time() - now)
        now = time.time()
    with open(fn) as fh:
        name_to_store = '/'.join(fn.split('/')[2:])
        output[name_to_store] = cleanhtml(fh.read())
        
records = pd.Series(output).to_frame('text')
overlap = map(lambda x: int(x[0] == x[1]), zip(records.iloc[0][0], records.iloc[1][0]))
cutoff = filter(lambda x: x[1] == 0, enumerate(overlap))[0][0]
records['text'] = records['text'].apply(lambda x: x[cutoff:])

records.to_csv('cablegate_parsed_records.csv')

# Appendix

Make sure to install something to read `.7z` files.

` brew install p7zip`

`pip install pylzma`